In [1]:
import os
import json
import pytz
import re
from datetime import datetime, timedelta
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

class MedicationSchedulerAgent:
    def __init__(self):
        self.SCOPES = ['https://www.googleapis.com/auth/calendar']
        self.CALENDAR_ID = os.getenv('CALENDAR_ID')
        self.CREDENTIALS_FILE = os.getenv('CREDENTIALS_FILE')
        self.TOKEN_FILE = os.getenv('TOKEN_FILE')
        self.KOREA_TZ = pytz.timezone('Asia/Seoul')
        self.service = self.authenticate()

    def authenticate(self):
        """구글 캘린더 API 인증"""
        creds = None
        
        if os.path.exists(self.TOKEN_FILE):
            try:
                creds = Credentials.from_authorized_user_file(self.TOKEN_FILE, self.SCOPES)
            except (json.JSONDecodeError, ValueError):
                print("토큰 파일이 손상되어 다시 인증합니다.")
                if os.path.exists(self.TOKEN_FILE):
                    os.remove(self.TOKEN_FILE)
                creds = None
        
        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(
                    self.CREDENTIALS_FILE, self.SCOPES)
                creds = flow.run_local_server(port=0)
            
            with open(self.TOKEN_FILE, 'w') as token:
                token.write(creds.to_json())
        
        return build('calendar', 'v3', credentials=creds)

    def parse_medication_text(self, patient_text):
        """약물 정보 텍스트 파싱"""
        medications = []
        
        # 다양한 패턴으로 약물 정보 추출
        patterns = {
            # 약물명과 복용 시간 패턴들
            'pattern1': r'(\w+)\s*(?:정|캡슐|포|알|mg|g)?\s*(?:\d+(?:\.\d+)?(?:mg|g)?)\s*[\s,]*(?:하루|일일)?\s*(\d+)\s*(?:회|번)\s*(?:복용|복용하세요|드시면)',
            'pattern2': r'(\w+)\s*(?:정|캡슐|포|알)?\s*[\s,]*(?:아침|점심|저녁|식전|식후|취침전)?\s*(?:하루|일일)?\s*(\d+)\s*(?:회|번)',
            'pattern3': r'(\w+)\s*[\s,]*(?:1일|하루)\s*(\d+)\s*(?:회|번)',
            'pattern4': r'(\w+).*?(\d+)\s*시간?\s*마다',
            'pattern5': r'(\w+).*?(?:아침|점심|저녁)',
        }
        
        # 시간 패턴
        time_patterns = {
            '아침': ['08:00', '아침 식사 후'],
            '점심': ['12:00', '점심 식사 후'], 
            '저녁': ['18:00', '저녁 식사 후'],
            '식전': ['07:30', '11:30', '17:30'],
            '식후': ['08:30', '12:30', '18:30'],
            '취침전': ['22:00', '취침 전'],
            '새벽': ['06:00', '새벽'],
        }
        
        # 텍스트를 줄 단위로 분석
        lines = patient_text.replace('\n', ' ').split('.')
        
        for line in lines:
            line = line.strip()
            if not line:
                continue
                
            # 각 패턴으로 매칭 시도
            for pattern_name, pattern in patterns.items():
                matches = re.finditer(pattern, line, re.IGNORECASE)
                
                for match in matches:
                    med_name = match.group(1).strip()
                    if len(med_name) < 2:  # 너무 짧은 이름은 제외
                        continue
                        
                    # 복용 횟수 추출
                    try:
                        frequency = int(match.group(2))
                    except:
                        frequency = 1
                    
                    # 복용 시간 결정
                    times = self.determine_medication_times(line, frequency)
                    
                    medications.append({
                        'name': med_name,
                        'frequency': frequency,
                        'times': times,
                        'original_text': line
                    })
        
        # 중복 제거 및 정리
        unique_medications = []
        seen_names = set()
        
        for med in medications:
            if med['name'] not in seen_names:
                unique_medications.append(med)
                seen_names.add(med['name'])
        
        return unique_medications

    def determine_medication_times(self, text, frequency):
        """텍스트 분석으로 복용 시간 결정"""
        times = []
        
        # 시간 키워드 매칭
        time_keywords = {
            '아침': '08:00',
            '점심': '12:00', 
            '저녁': '18:00',
            '취침': '22:00',
            '식전': ['07:30', '11:30', '17:30'],
            '식후': ['08:30', '12:30', '18:30']
        }
        
        text_lower = text.lower()
        
        # 키워드별 시간 추출
        for keyword, time_list in time_keywords.items():
            if keyword in text_lower:
                if isinstance(time_list, list):
                    if frequency <= len(time_list):
                        times.extend(time_list[:frequency])
                    else:
                        times.extend(time_list)
                else:
                    times.append(time_list)
        
        # 시간이 없으면 기본값 설정
        if not times:
            default_times = {
                1: ['08:00'],  # 하루 1회
                2: ['08:00', '20:00'],  # 하루 2회
                3: ['08:00', '12:00', '20:00'],  # 하루 3회
                4: ['08:00', '12:00', '16:00', '20:00']  # 하루 4회
            }
            times = default_times.get(frequency, ['08:00', '12:00', '20:00'])
        
        return times[:frequency]  # 복용 횟수만큼만 반환

    def create_medication_schedule(self, medications, start_date=None, duration_days=30):
        """약물 복용 일정을 캘린더에 추가"""
        if start_date is None:
            start_date = datetime.now(self.KOREA_TZ).date()
        
        created_events = []
        
        print(f"\n🏥 약물 복용 일정을 생성합니다...")
        print(f"📅 시작일: {start_date}")
        print(f"⏱️  기간: {duration_days}일")
        print("="*50)
        
        for med in medications:
            print(f"\n💊 {med['name']} - 하루 {med['frequency']}회")
            print(f"⏰ 복용 시간: {', '.join(med['times'])}")
            
            # 각 복용 시간별로 일정 생성
            for time_str in med['times']:
                # 반복 일정으로 생성
                success = self.create_recurring_medication_event(
                    med_name=med['name'],
                    time_str=time_str,
                    start_date=start_date,
                    duration_days=duration_days
                )
                
                if success:
                    created_events.append(f"{med['name']} at {time_str}")
                    print(f"  ✅ {time_str} 일정 생성 완료")
                else:
                    print(f"  ❌ {time_str} 일정 생성 실패")
        
        return created_events

    def create_recurring_medication_event(self, med_name, time_str, start_date, duration_days):
        """반복되는 약물 복용 일정 생성"""
        try:
            # 시작 날짜와 시간 결합
            hour, minute = map(int, time_str.split(':'))
            start_datetime = datetime.combine(start_date, datetime.min.time().replace(hour=hour, minute=minute))
            start_datetime = self.KOREA_TZ.localize(start_datetime)
            
            # 종료 시간 (30분 후)
            end_datetime = start_datetime + timedelta(minutes=30)
            
            # 종료 날짜
            end_date = start_date + timedelta(days=duration_days)
            
            # 이벤트 생성
            event = {
                'summary': f'💊 {med_name} 복용',
                'description': f'약물: {med_name}\n복용 시간: {time_str}\n⚠️ 정확한 시간에 복용하세요!',
                'start': {
                    'dateTime': start_datetime.isoformat(),
                    'timeZone': 'Asia/Seoul',
                },
                'end': {
                    'dateTime': end_datetime.isoformat(),
                    'timeZone': 'Asia/Seoul',
                },
                'recurrence': [
                    f'RRULE:FREQ=DAILY;UNTIL={end_date.strftime("%Y%m%d")}T235959Z'
                ],
                'reminders': {
                    'useDefault': False,
                    'overrides': [
                        {'method': 'popup', 'minutes': 10},  # 10분 전 알림
                        {'method': 'popup', 'minutes': 0},   # 정시 알림
                    ],
                },
            }
            
            created_event = self.service.events().insert(
                calendarId=self.CALENDAR_ID,
                body=event
            ).execute()
            
            return True
            
        except HttpError as error:
            print(f'일정 생성 중 오류 발생: {error}')
            return False

    def process_patient_text(self, patient_text):
        """환자 텍스트를 처리하여 약물 일정 생성"""
        print("🤖 약물 정보 분석 에이전트 시작...")
        print("="*60)
        print("📄 받은 텍스트:")
        print("-" * 30)
        print(patient_text)
        print("-" * 30)
        
        # 1. 텍스트 파싱
        medications = self.parse_medication_text(patient_text)
        
        if not medications:
            print("❌ 약물 정보를 찾을 수 없습니다.")
            print("💡 텍스트에 다음과 같은 정보가 포함되어야 합니다:")
            print("   - 약물명")
            print("   - 복용 횟수 (예: 하루 3회)")
            print("   - 복용 시간 (예: 아침, 점심, 저녁)")
            return []
        
        print(f"\n✅ {len(medications)}개의 약물 정보를 찾았습니다:")
        for i, med in enumerate(medications, 1):
            print(f"{i}. {med['name']} - 하루 {med['frequency']}회")
            print(f"   복용시간: {', '.join(med['times'])}")
        
        # 2. 사용자 확인
        confirm = input(f"\n위 정보로 일정을 생성하시겠습니까? (Y/n): ").strip().lower()
        if confirm == 'n':
            print("일정 생성을 취소합니다.")
            return []
        
        # 3. 기간 설정
        duration_input = input("복용 기간 (일수, 기본값: 30일): ").strip()
        duration_days = int(duration_input) if duration_input.isdigit() else 30
        
        # 4. 일정 생성
        created_events = self.create_medication_schedule(medications, duration_days=duration_days)
        
        print(f"\n🎉 총 {len(created_events)}개의 약물 복용 일정이 생성되었습니다!")
        return created_events


def main():
    """메인 프로그램"""
    try:
        agent = MedicationSchedulerAgent()
        print("🎉 약물 복용 일정 에이전트 연결 성공!")
    except Exception as e:
        print(f"❌ 초기화 오류: {e}")
        return
    
    while True:
        print("\n" + "="*60)
        print("💊 약물 복용 일정 자동 생성 에이전트")
        print("="*60)
        print("1. 📝 약물 정보 텍스트 입력")
        print("2. 📁 텍스트 파일에서 불러오기")
        print("3. 📋 생성된 약물 일정 확인")
        print("4. 🗑️  약물 일정 삭제")
        print("5. 🚪 종료")
        
        choice = input("\n선택하세요 (1-5): ").strip()
        
        if choice == '1':
            # 직접 텍스트 입력
            print("\n📄 약물 정보를 입력하세요:")
            print("예시: 타이레놀 500mg 하루 3회 복용, 아침 점심 저녁 식후")
            print("여러 줄 입력 가능 (끝내려면 빈 줄에서 Enter):")
            
            lines = []
            while True:
                line = input()
                if line.strip() == "":
                    break
                lines.append(line)
            
            patient_text = '\n'.join(lines)
            
            if patient_text.strip():
                agent.process_patient_text(patient_text)
            else:
                print("❌ 텍스트가 입력되지 않았습니다.")
        
        elif choice == '2':
            # 파일에서 불러오기
            filename = input("파일 경로를 입력하세요: ").strip()
            try:
                with open(filename, 'r', encoding='utf-8') as f:
                    patient_text = f.read()
                
                if patient_text.strip():
                    agent.process_patient_text(patient_text)
                else:
                    print("❌ 파일이 비어있습니다.")
            except FileNotFoundError:
                print("❌ 파일을 찾을 수 없습니다.")
            except Exception as e:
                print(f"❌ 파일 읽기 오류: {e}")
        
        elif choice == '3':
            # 약물 일정 확인
            try:
                now_kst = datetime.now(agent.KOREA_TZ)
                events_result = agent.service.events().list(
                    calendarId=agent.CALENDAR_ID,
                    timeMin=now_kst.isoformat(),
                    q='💊',  # 약물 이모지로 검색
                    maxResults=50,
                    singleEvents=True,
                    orderBy='startTime'
                ).execute()
                
                events = events_result.get('items', [])
                
                if not events:
                    print("💊 약물 복용 일정이 없습니다.")
                else:
                    print(f"\n💊 약물 복용 일정 ({len(events)}개):")
                    for i, event in enumerate(events, 1):
                        start = event['start'].get('dateTime', event['start'].get('date'))
                        try:
                            dt = datetime.fromisoformat(start.replace('Z', '+00:00'))
                            dt_kst = dt.astimezone(agent.KOREA_TZ)
                            start_display = dt_kst.strftime('%m/%d %H:%M')
                        except:
                            start_display = start
                        
                        print(f"{i:2d}. {event['summary']} - {start_display}")
            
            except HttpError as error:
                print(f'일정 조회 중 오류 발생: {error}')
        
        elif choice == '4':
            # 약물 일정 삭제
            confirm = input("⚠️  모든 약물 복용 일정을 삭제하시겠습니까? (y/N): ").strip().lower()
            if confirm == 'y':
                try:
                    # 약물 일정 검색
                    events_result = agent.service.events().list(
                        calendarId=agent.CALENDAR_ID,
                        q='💊',
                        maxResults=500
                    ).execute()
                    
                    events = events_result.get('items', [])
                    deleted_count = 0
                    
                    for event in events:
                        try:
                            agent.service.events().delete(
                                calendarId=agent.CALENDAR_ID,
                                eventId=event['id']
                            ).execute()
                            deleted_count += 1
                        except:
                            pass
                    
                    print(f"✅ {deleted_count}개의 약물 일정이 삭제되었습니다.")
                
                except HttpError as error:
                    print(f'일정 삭제 중 오류 발생: {error}')
            else:
                print("삭제가 취소되었습니다.")
        
        elif choice == '5':
            print("👋 약물 복용 일정 에이전트를 종료합니다.")
            break
        
        else:
            print("❌ 올바르지 않은 선택입니다. 1-5 중에서 선택해주세요.")


# 에이전트 함수로 직접 호출 가능
def create_medication_schedule_from_text(patient_text, duration_days=30):
    """외부에서 호출 가능한 에이전트 함수"""
    try:
        agent = MedicationSchedulerAgent()
        return agent.process_patient_text(patient_text)
    except Exception as e:
        print(f"에러: {e}")
        return []


if __name__ == '__main__':
    main()


🎉 약물 복용 일정 에이전트 연결 성공!

💊 약물 복용 일정 자동 생성 에이전트
1. 📝 약물 정보 텍스트 입력
2. 📁 텍스트 파일에서 불러오기
3. 📋 생성된 약물 일정 확인
4. 🗑️  약물 일정 삭제
5. 🚪 종료
👋 약물 복용 일정 에이전트를 종료합니다.
